In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# df10=pd.read_csv('/kaggle/input/ecommerce-behavior-data-from-multi-category-store/2019-Oct.csv',usecols=['event_time','event_type','product_id','price'])
# df11=pd.read_csv('/kaggle/input/ecommerce-behavior-data-from-multi-category-store/2019-Nov.csv',usecols=['event_time','event_type','product_id','price'])
# df=pd.concat([df10,df11]).reset_index(drop=True)
# del df10
# del df11
# price_summary=df[['product_id','price']].drop_duplicates().groupby('product_id').agg({'price':'count'}).sort_values('price',ascending=False).reset_index()
# morethan100p=list(price_summary[price_summary.price>=100].product_id)
# df100=df[df.product_id.isin(morethan100p)]
# df100.to_csv('/kaggle/working/df100.csv',index=False)

In [ ]:
df=pd.read_csv('/kaggle/input/ecommerce/df100.csv')

In [ ]:
df['event_time']=df['event_time'].str[:10]

In [ ]:
df.head()

In [ ]:
df['week_id']=pd.to_datetime(df.event_time).dt.week
df.head()

In [ ]:
purchase=df[df.event_type=='purchase'].reset_index(drop=True)
cart=df[df.event_type=='cart'].reset_index(drop=True)
view=df[df.event_type=='view'].reset_index(drop=True)

In [ ]:
demand_chart=purchase.groupby(['product_id','event_time']).agg({'event_type':'count','price':'mean','week_id':'max'}).reset_index().rename(columns={"event_time":'date',"event_type":'sales',"price":'avg_price'})
demand_chart=demand_chart.groupby(['product_id','week_id']).agg({'date':'count','avg_price':'mean','sales':'sum'}).reset_index().rename(columns={"week_id":'week_id',"event_type":'sales',"avg_price":'avg_price','date':'n_days'})

In [ ]:
demand_chart['sales']=demand_chart['sales']/demand_chart['n_days']*7
demand_chart

In [ ]:
demand_chart['previous_week_id']=demand_chart['week_id']-1

In [ ]:
elasticity_df=demand_chart.merge(demand_chart[['product_id','week_id','sales','avg_price']],how='left',left_on=['product_id','previous_week_id'],right_on=['product_id','week_id']).dropna()

In [ ]:
elasticity_df[elasticity_df.product_id==1004870]

In [ ]:
from sklearn.linear_model import LinearRegression,RANSACRegressor,HuberRegressor
from sklearn.metrics import r2_score
import statsmodels.api as sm
elasticity_dicts=[]
for product_id in elasticity_df.product_id.unique():
    entry={}
    entry_df=elasticity_df[elasticity_df.product_id==product_id]
#     y=entry_df.sales_x-entry_df.sales_y
#     X=pd.DataFrame((entry_df.avg_price_x-entry_df.avg_price_y)*entry_df.sales_y/entry_df.avg_price_y)
    y=np.log(entry_df.sales_x)
    X=pd.DataFrame(np.log(entry_df.avg_price_x))
#     lr=HuberRegressor()
#     lr.fit(X,y)
    X = sm.add_constant(X)
    model = sm.OLS(y, X)
    result = model.fit()
    
    rsquared = result.rsquared
    coefficient_pvalue = result.f_pvalue
    try:
        intercept, slope = result.params
    except:
        intercept, slope=0,0
    mean_price = np.mean(X)
    mean_quantity = np.mean(y)
#     tintercept, t_score = result.tvalues
#     if coefficient_pvalue<0.05:
    print(product_id,':','slope:',slope,'R2 Score:',rsquared,'coefficient_pvalue:',coefficient_pvalue)
    entry['product']=product_id
    entry['elasticity']=slope
    entry['R2 Score']=rsquared
    entry['coefficient_pvalue']=coefficient_pvalue
    elasticity_dicts.append(entry)
    
#     else:
#         print(product_id,'p_value smaller than 0.05')
#     print(product_id,':',lr.coef_,'R2_score:',r2_score(y,lr.predict(X)))
elasticity_chart=pd.DataFrame(elasticity_dicts)

In [ ]:
elasticity_chart

In [ ]:
def fix_elasticity(x):
    if x.coefficient_pvalue<=0.05:
        return x.elasticity
    else:
        return 0
elasticity_chart['elasticity_fixed']=elasticity_chart.apply(fix_elasticity,axis=1)
elasticity_chart['elasticity_fixed']=elasticity_chart.elasticity_fixed.apply(lambda x: x if x <0 else 0)

In [ ]:
elasticity_chart

In [ ]:
purchase_df=purchase.groupby(['product_id','week_id']).agg({'price':'mean','event_type':'count'}).reset_index().rename(columns={'event_type':'sales'})
purchase_df['sales_last_1w']=purchase_df.groupby('product_id')['sales'].rolling(2).sum().reset_index()['sales']-purchase_df['sales']
purchase_df['sales_last_2w']=purchase_df.groupby('product_id')['sales'].rolling(3).sum().reset_index()['sales']-purchase_df['sales']
purchase_df['sales_last_4w']=purchase_df.groupby('product_id')['sales'].rolling(5).sum().reset_index()['sales']-purchase_df['sales']
purchase_df

In [ ]:
view_df=view.groupby(['product_id','week_id']).agg({'event_type':'count'}).reset_index().rename(columns={'event_type':'views'})
view_df['views_last_1w']=view_df.groupby('product_id')['views'].rolling(2).sum().reset_index()['views']-view_df['views']
view_df['views_last_2w']=view_df.groupby('product_id')['views'].rolling(3).sum().reset_index()['views']-view_df['views']
view_df['views_last_4w']=view_df.groupby('product_id')['views'].rolling(5).sum().reset_index()['views']-view_df['views']
view_df

In [ ]:
view_df[view_df.product_id==7006302]

In [ ]:
cart_df=cart.groupby(['product_id','week_id']).agg({'event_type':'count'}).reset_index().rename(columns={'event_type':'carts'})
cart_df['carts_last_1w']=cart_df.groupby('product_id')['carts'].rolling(2).sum().reset_index()['carts']-cart_df['carts']
cart_df['carts_last_2w']=cart_df.groupby('product_id')['carts'].rolling(3).sum().reset_index()['carts']-cart_df['carts']
cart_df['carts_last_4w']=cart_df.groupby('product_id')['carts'].rolling(5).sum().reset_index()['carts']-cart_df['carts']
cart_df


In [ ]:
master=purchase_df.merge(view_df,on=['product_id','week_id'],how='left').merge(cart_df,on=['product_id','week_id'],how='left')
master=master[['week_id','product_id','sales','sales_last_1w','sales_last_2w','sales_last_4w','views_last_1w','views_last_2w','views_last_4w','carts_last_1w','carts_last_2w','carts_last_4w']]
master['sales_1w2w_ratio']=master.sales_last_1w/master.sales_last_2w
master['sales_2w2w_ratio']=master.sales_last_2w/master.sales_last_4w
master['sales_1w4w_ratio']=master.sales_last_1w/master.sales_last_4w
master['views_1w2w_ratio']=master.views_last_1w/master.views_last_2w
master['views_2w2w_ratio']=master.views_last_2w/master.views_last_4w
master['views_1w4w_ratio']=master.views_last_1w/master.views_last_4w
master['carts_1w2w_ratio']=master.carts_last_1w/master.carts_last_2w
master['carts_2w2w_ratio']=master.carts_last_2w/master.carts_last_4w
master['carts_1w4w_ratio']=master.carts_last_1w/master.carts_last_4w

In [ ]:
print(list(master))

In [ ]:
for e in ['sales','sales_last_1w','sales_last_2w','sales_last_4w','views_last_1w','views_last_2w','views_last_4w',
          'carts_last_1w','carts_last_2w','carts_last_4w']:
    master[e]=np.log(master[e])

In [ ]:
train=master[master.week_id.isin([44,45,46,47])]
test=master[master.week_id.isin([48])]


In [ ]:
features=['sales_last_1w', 'sales_last_2w', 'sales_last_4w', 'views_last_1w', 'views_last_2w', 'views_last_4w', 'carts_last_1w', 'carts_last_2w', 'carts_last_4w', 'sales_1w2w_ratio', 'sales_2w2w_ratio', 'sales_1w4w_ratio', 'views_1w2w_ratio', 'views_2w2w_ratio', 'views_1w4w_ratio', 'carts_1w2w_ratio', 'carts_2w2w_ratio', 'carts_1w4w_ratio']

In [ ]:
import xgboost as xgb

clf=xgb.XGBRegressor(n_estimators=1000,learning_rate=0.005)
clf.fit(train[features],train['sales'],eval_set=[(train[features],train['sales']),(test[features],test['sales'])],eval_metric='rmse',verbose=True,early_stopping_rounds=20)


In [ ]:
test['prediction']=clf.predict(test[features])

In [ ]:
test['sales_real']=np.exp(test['sales'])
test['sales_prediction']=np.exp(test['prediction']).astype('int')

In [ ]:
test.head(60)

In [ ]:
estimate=test[['week_id','product_id','sales_prediction']].merge(purchase_df[['week_id','product_id','sales','price']],how='left',on=['product_id','week_id'])
estimate=estimate.merge(elasticity_chart,how='left',left_on='product_id',right_on='product')

In [ ]:
estimate

In [ ]:
from scipy.optimize import minimize
results=[]
for product_id in estimate.product_id.unique():
    entry={}
    entry['product_id']=product_id
    
    sample=estimate[estimate.product_id==product_id]
    

    lowest=int(sample.price*0.5)
    highest=int(sample.price*1.5)
    
    sales_prediction=sample.sales_prediction
    elasticity=sample.elasticity_fixed
    price=sample.price

    con1 = {'type': 'ineq', 'fun': lambda x:x-lowest}
    con2 = {'type': 'ineq', 'fun': lambda x:highest-x}
    cons = ([con1,con2])

    fun = lambda x: -1*(sales_prediction*elasticity/price*x**2 - x*sales_prediction*elasticity +x*sales_prediction)

    res = minimize(fun,x0=0,method='SLSQP',constraints=cons)
    entry['optimized_price']=res.x[0]
    results.append(entry)

In [ ]:
estimate=estimate.merge(pd.DataFrame(results),how='left',on='product_id')

In [ ]:
estimate.head(60)